In [1]:
import os
import glob
import zipfile
import pandas as pd

In [2]:
import pyspark

In [3]:
os.getcwd()

'/Users/jill/Downloads/2020_Project/EDA'

In [203]:
ini_path = "/Users/jill/Downloads/2020_Project"

In [204]:
sample_path= ini_path+"/DE1.0_Data"

In [129]:
file_lst = [f for f in os.listdir(data_path) if f[0]==("s")]

In [130]:
len(file_lst)

20

In [205]:
def file_lst (datapath,filetype):
    lst = []
    for filedirs, files, filenames in os.walk(datapath):
        lst += [os.path.join(filedirs, file) for file in filenames if file.endswith(filetype)]
    return lst

In [208]:
ziptype = ".zip"
filelst = file_lst(sample_path,ziptype)

In [209]:
len (filelst)

160

In [67]:
filelst

['/Users/jill/Downloads/2020_Project/DE1.0_Data/sample_5/176565_DE1_0_2008_to_2010_Outpatient_Claims_Sample_5.zip',
 '/Users/jill/Downloads/2020_Project/DE1.0_Data/sample_5/176539_DE1_0_2009_Beneficiary_Summary_File_Sample_5.zip',
 '/Users/jill/Downloads/2020_Project/DE1.0_Data/sample_5/DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_5.zip',
 '/Users/jill/Downloads/2020_Project/DE1.0_Data/sample_5/176619_DE1_0_2008_Beneficiary_Summary_File_Sample_5.zip',
 '/Users/jill/Downloads/2020_Project/DE1.0_Data/sample_5/176580_DE1_0_2010_Beneficiary_Summary_File_Sample_5.zip',
 '/Users/jill/Downloads/2020_Project/DE1.0_Data/sample_5/176577_DE1_0_2008_to_2010_Inpatient_Claims_Sample_5.zip',
 '/Users/jill/Downloads/2020_Project/DE1.0_Data/sample_5/DE1_0_2008_to_2010_Carrier_Claims_Sample_5B.zip',
 '/Users/jill/Downloads/2020_Project/DE1.0_Data/sample_5/DE1_0_2008_to_2010_Carrier_Claims_Sample_5A.zip',
 '/Users/jill/Downloads/2020_Project/DE1.0_Data/sample_2/zip/176629_DE1_0_2009_Beneficiary_Sum

In [134]:
benefician_lst = [f for f in filelst if "Beneficiary_Summary" in f]

In [135]:
len (benefician_lst)

60

In [167]:
def sublst(lstname,filelst):
    return [f for f in filelst if lstname in f]
        

In [137]:
benefician_lst,inpatient_lst,outpatient_lst,carrier_lst,pde_lst = sublst("Beneficiary_Summary"),sublst("Inpatient"),sublst("Outpatient"),sublst("Carrier"),sublst("Prescription")

In [138]:
len(benefician_lst),len(inpatient_lst),len(outpatient_lst),len(carrier_lst),len(pde_lst)

(60, 20, 20, 40, 20)

In [139]:
def unzip_file(filepath):
    for f in filepath:
        zip_obj = zipfile.ZipFile(f,'r')
        zip_obj.extractall(outpath)
        zip_obj.close()
    

In [140]:
outpath = startpath+"/integrated_data/Beneficiary_Summary"
unzip_file(benefician_lst)

In [141]:
sub_lst = [benefician_lst,inpatient_lst,outpatient_lst,carrier_lst,pde_lst]

In [142]:
def unzip_file(startpath,sub_lst,subfolder):
    for i in range(5):
        outpath = startpath+subfolder[i]
        filepath = sub_lst[i]
        for f in filepath:
            zip_obj = zipfile.ZipFile(f,'r')
            zip_obj.extractall(outpath)
    zip_obj.close()     

In [145]:
# Unzip the files to out path. 
sub_lst = [benefician_lst,inpatient_lst,outpatient_lst,carrier_lst,pde_lst]
subfolder = ['Beneficiary_Summary','inpatient_claim','outpatient_claim',"carrier_claim","pde_claim"]
startpath = "/Users/jill/Downloads/2020_Project/integrated_data/"
unzip_file(startpath,sub_lst,subfolder)

## 2. Concat dataset

In [3]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
startpath = "/Users/jill/Downloads/2020_Project/integrated_data"
file_location = startpath+"/inpatient_claim/DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv"
df = spark.read.format("csv").option("inferSchema", 
           True).option("header", True).load(file_location)
display(df)

DataFrame[DESYNPUF_ID: string, CLM_ID: bigint, SEGMENT: int, CLM_FROM_DT: int, CLM_THRU_DT: int, PRVDR_NUM: string, CLM_PMT_AMT: double, NCH_PRMRY_PYR_CLM_PD_AMT: double, AT_PHYSN_NPI: bigint, OP_PHYSN_NPI: bigint, OT_PHYSN_NPI: bigint, CLM_ADMSN_DT: int, ADMTNG_ICD9_DGNS_CD: string, CLM_PASS_THRU_PER_DIEM_AMT: double, NCH_BENE_IP_DDCTBL_AMT: double, NCH_BENE_PTA_COINSRNC_LBLTY_AM: double, NCH_BENE_BLOOD_DDCTBL_LBLTY_AM: double, CLM_UTLZTN_DAY_CNT: int, NCH_BENE_DSCHRG_DT: int, CLM_DRG_CD: string, ICD9_DGNS_CD_1: string, ICD9_DGNS_CD_2: string, ICD9_DGNS_CD_3: string, ICD9_DGNS_CD_4: string, ICD9_DGNS_CD_5: string, ICD9_DGNS_CD_6: string, ICD9_DGNS_CD_7: string, ICD9_DGNS_CD_8: string, ICD9_DGNS_CD_9: string, ICD9_DGNS_CD_10: string, ICD9_PRCDR_CD_1: int, ICD9_PRCDR_CD_2: string, ICD9_PRCDR_CD_3: string, ICD9_PRCDR_CD_4: string, ICD9_PRCDR_CD_5: string, ICD9_PRCDR_CD_6: string, HCPCS_CD_1: string, HCPCS_CD_2: string, HCPCS_CD_3: string, HCPCS_CD_4: string, HCPCS_CD_5: string, HCPCS_CD_

In [5]:
df.show(truncate=0)

+----------------+---------------+-------+-----------+-----------+---------+-----------+------------------------+------------+------------+------------+------------+-------------------+--------------------------+----------------------+------------------------------+------------------------------+------------------+------------------+----------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------

In [6]:
df1 = df.toPandas()

In [7]:
df1.shape

(66773, 81)

In [8]:
file_location = startpath+"/inpatient_claim/DE1_0_2008_to_2010_Inpatient_Claims_Sample_2.csv"
df_2 = spark.read.format("csv").option("inferSchema", 
           True).option("header", True).load(file_location)
display(df_2)

DataFrame[DESYNPUF_ID: string, CLM_ID: bigint, SEGMENT: int, CLM_FROM_DT: int, CLM_THRU_DT: int, PRVDR_NUM: string, CLM_PMT_AMT: double, NCH_PRMRY_PYR_CLM_PD_AMT: double, AT_PHYSN_NPI: bigint, OP_PHYSN_NPI: bigint, OT_PHYSN_NPI: bigint, CLM_ADMSN_DT: int, ADMTNG_ICD9_DGNS_CD: string, CLM_PASS_THRU_PER_DIEM_AMT: double, NCH_BENE_IP_DDCTBL_AMT: double, NCH_BENE_PTA_COINSRNC_LBLTY_AM: double, NCH_BENE_BLOOD_DDCTBL_LBLTY_AM: double, CLM_UTLZTN_DAY_CNT: int, NCH_BENE_DSCHRG_DT: int, CLM_DRG_CD: string, ICD9_DGNS_CD_1: string, ICD9_DGNS_CD_2: string, ICD9_DGNS_CD_3: string, ICD9_DGNS_CD_4: string, ICD9_DGNS_CD_5: string, ICD9_DGNS_CD_6: string, ICD9_DGNS_CD_7: string, ICD9_DGNS_CD_8: string, ICD9_DGNS_CD_9: string, ICD9_DGNS_CD_10: string, ICD9_PRCDR_CD_1: int, ICD9_PRCDR_CD_2: string, ICD9_PRCDR_CD_3: string, ICD9_PRCDR_CD_4: string, ICD9_PRCDR_CD_5: string, ICD9_PRCDR_CD_6: string, HCPCS_CD_1: string, HCPCS_CD_2: string, HCPCS_CD_3: string, HCPCS_CD_4: string, HCPCS_CD_5: string, HCPCS_CD_

In [9]:
df2 = df_2.toPandas()

In [10]:
df2.shape

(66494, 81)

In [11]:
df2.shape[0]+df1.shape[0]

133267

In [12]:
df1.columns.tolist() ==df2.columns.tolist()

True

In [13]:
df_3 = df.union(df_2)

In [31]:
df_3.write.save("/Users/jill/Downloads/2020_Project/integrated_data/inpatient_claim/df_3")

In [14]:
df3 = df_3.toPandas()

In [32]:
df4 = spark.read.load("/Users/jill/Downloads/2020_Project/integrated_data/inpatient_claim/df_3")
display(df4)

DataFrame[DESYNPUF_ID: string, CLM_ID: bigint, SEGMENT: int, CLM_FROM_DT: int, CLM_THRU_DT: int, PRVDR_NUM: string, CLM_PMT_AMT: double, NCH_PRMRY_PYR_CLM_PD_AMT: double, AT_PHYSN_NPI: bigint, OP_PHYSN_NPI: bigint, OT_PHYSN_NPI: bigint, CLM_ADMSN_DT: int, ADMTNG_ICD9_DGNS_CD: string, CLM_PASS_THRU_PER_DIEM_AMT: double, NCH_BENE_IP_DDCTBL_AMT: double, NCH_BENE_PTA_COINSRNC_LBLTY_AM: double, NCH_BENE_BLOOD_DDCTBL_LBLTY_AM: double, CLM_UTLZTN_DAY_CNT: int, NCH_BENE_DSCHRG_DT: int, CLM_DRG_CD: string, ICD9_DGNS_CD_1: string, ICD9_DGNS_CD_2: string, ICD9_DGNS_CD_3: string, ICD9_DGNS_CD_4: string, ICD9_DGNS_CD_5: string, ICD9_DGNS_CD_6: string, ICD9_DGNS_CD_7: string, ICD9_DGNS_CD_8: string, ICD9_DGNS_CD_9: string, ICD9_DGNS_CD_10: string, ICD9_PRCDR_CD_1: int, ICD9_PRCDR_CD_2: string, ICD9_PRCDR_CD_3: string, ICD9_PRCDR_CD_4: string, ICD9_PRCDR_CD_5: string, ICD9_PRCDR_CD_6: string, HCPCS_CD_1: string, HCPCS_CD_2: string, HCPCS_CD_3: string, HCPCS_CD_4: string, HCPCS_CD_5: string, HCPCS_CD_

In [34]:
df_4 = df4.toPandas()

In [35]:
df_4.shape

(133267, 81)

In [33]:
df4.show()

+----------------+--------------+-------+-----------+-----------+---------+-----------+------------------------+------------+------------+------------+------------+-------------------+--------------------------+----------------------+------------------------------+------------------------------+------------------+------------------+----------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+---------

In [25]:
df3.shape

(133267, 81)

In [26]:
startpath = "/Users/jill/Downloads/2020_Project/integrated_data/inpatient_claim/"
datapath_lst = [os.path.join(startpath, file) for file in os.listdir(startpath) if file.endswith("csv")]

In [27]:
datapath_lst[0]

'/Users/jill/Downloads/2020_Project/integrated_data/inpatient_claim/DE1_0_2008_to_2010_Inpatient_Claims_Sample_13.csv'

In [15]:
def datapath_lst(startpath):
    datapath_lst = [os.path.join(startpath, file) for file in os.listdir(startpath)]
    return datapath_lst

In [16]:
def file_lst (datapath,filetype):
    lst = []
    for filedirs, files, filenames in os.walk(datapath):
        lst += [os.path.join(filedirs, file) for file in filenames if file.endswith(filetype)]
    return lst

In [30]:
def concat_df(datapath):
    datapath_lst = [os.path.join(datapath, file) for file in os.listdir(datapath) if file.endswith("csv")]
    df1 = spark.read.format("csv").option("inferSchema", True).option("header", True).load(datapath_lst[0])
    for i in range(1,20):
        df2 = spark.read.format("csv").option("inferSchema", True).option("header", True).load(datapath_lst[i])
        df1 = df1.union(df2) 
    return df1   

## Inpatient concat data

In [31]:

startpath_in = "/Users/jill/Downloads/2020_Project/integrated_data/inpatient_claim/"
df_in = concat_df(startpath_in)

In [32]:
df_in.write.save("./concat_data/inpatient", format='parquet')

In [8]:
df_inpatient = spark.read.load("/Users/jill/Downloads/2020_Project/EDA/concat_data/inpatient/inpatient_con/")

In [38]:
df_inpatient.take(5)

[Row(DESYNPUF_ID='4164404F0A50EC8D', CLM_ID=90191100136835, SEGMENT=1, CLM_FROM_DT=20080123, CLM_THRU_DT=20080127, PRVDR_NUM='3901ZS', CLM_PMT_AMT=2000.0, NCH_PRMRY_PYR_CLM_PD_AMT=0.0, AT_PHYSN_NPI=771958540, OP_PHYSN_NPI=7718505294, OT_PHYSN_NPI=None, CLM_ADMSN_DT=20080123, ADMTNG_ICD9_DGNS_CD='2910', CLM_PASS_THRU_PER_DIEM_AMT=0.0, NCH_BENE_IP_DDCTBL_AMT=1024.0, NCH_BENE_PTA_COINSRNC_LBLTY_AM=0.0, NCH_BENE_BLOOD_DDCTBL_LBLTY_AM=0.0, CLM_UTLZTN_DAY_CNT=4, NCH_BENE_DSCHRG_DT=20080127, CLM_DRG_CD='895', ICD9_DGNS_CD_1='29181', ICD9_DGNS_CD_2='7850', ICD9_DGNS_CD_3='30560', ICD9_DGNS_CD_4='V1046', ICD9_DGNS_CD_5='4019', ICD9_DGNS_CD_6='30390', ICD9_DGNS_CD_7='40390', ICD9_DGNS_CD_8='2920', ICD9_DGNS_CD_9='2967', ICD9_DGNS_CD_10=None, ICD9_PRCDR_CD_1=586, ICD9_PRCDR_CD_2='2768', ICD9_PRCDR_CD_3=None, ICD9_PRCDR_CD_4=None, ICD9_PRCDR_CD_5=None, ICD9_PRCDR_CD_6=None, HCPCS_CD_1=None, HCPCS_CD_2=None, HCPCS_CD_3=None, HCPCS_CD_4=None, HCPCS_CD_5=None, HCPCS_CD_6=None, HCPCS_CD_7=None, HCPCS_

In [157]:
df_inpatient.printSchema()

root
 |-- DESYNPUF_ID: string (nullable = true)
 |-- CLM_ID: long (nullable = true)
 |-- SEGMENT: integer (nullable = true)
 |-- CLM_FROM_DT: integer (nullable = true)
 |-- CLM_THRU_DT: integer (nullable = true)
 |-- PRVDR_NUM: string (nullable = true)
 |-- CLM_PMT_AMT: double (nullable = true)
 |-- NCH_PRMRY_PYR_CLM_PD_AMT: double (nullable = true)
 |-- AT_PHYSN_NPI: long (nullable = true)
 |-- OP_PHYSN_NPI: long (nullable = true)
 |-- OT_PHYSN_NPI: long (nullable = true)
 |-- CLM_ADMSN_DT: integer (nullable = true)
 |-- ADMTNG_ICD9_DGNS_CD: string (nullable = true)
 |-- CLM_PASS_THRU_PER_DIEM_AMT: double (nullable = true)
 |-- NCH_BENE_IP_DDCTBL_AMT: double (nullable = true)
 |-- NCH_BENE_PTA_COINSRNC_LBLTY_AM: double (nullable = true)
 |-- NCH_BENE_BLOOD_DDCTBL_LBLTY_AM: double (nullable = true)
 |-- CLM_UTLZTN_DAY_CNT: integer (nullable = true)
 |-- NCH_BENE_DSCHRG_DT: integer (nullable = true)
 |-- CLM_DRG_CD: string (nullable = true)
 |-- ICD9_DGNS_CD_1: string (nullable = true)


In [43]:
df_inpatient.show(1)

+----------------+--------------+-------+-----------+-----------+---------+-----------+------------------------+------------+------------+------------+------------+-------------------+--------------------------+----------------------+------------------------------+------------------------------+------------------+------------------+----------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+---------

In [158]:
df_inpatient.count()

1332822

In [45]:
df_inpatient.filter(df_inpatient["ICD9_DGNS_CD_1"] == "1830").show()

+----------------+---------------+-------+-----------+-----------+---------+-----------+------------------------+------------+------------+------------+------------+-------------------+--------------------------+----------------------+------------------------------+------------------------------+------------------+------------------+----------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------

In [81]:
df_inpatient_OC = df_inpatient.filter((df_inpatient["ICD9_DGNS_CD_1"] == "1830")|
                                      (df_inpatient["ICD9_DGNS_CD_2"] == "1830")|
                                      (df_inpatient["ICD9_DGNS_CD_3"] == "1830")|
                                      (df_inpatient["ICD9_DGNS_CD_4"] == "1830")|
                                      (df_inpatient["ICD9_DGNS_CD_5"] == "1830")|
                                      (df_inpatient["ICD9_DGNS_CD_6"] == "1830")|
                                      (df_inpatient["ICD9_DGNS_CD_7"] == "1830")|
                                      (df_inpatient["ICD9_DGNS_CD_8"] == "1830")|
                                      (df_inpatient["ICD9_DGNS_CD_9"] == "1830")|
                                      (df_inpatient["ICD9_DGNS_CD_10"] == "1830")
                                     )

In [82]:
df_inpatient_OC.count()

2649

In [161]:
df0000 = df_inpatient.where(df_inpatient["ICD9_DGNS_CD_1"].eqNullSafe('1830')|
                            df_inpatient["ICD9_DGNS_CD_2"].eqNullSafe('1830')|
                            df_inpatient["ICD9_DGNS_CD_3"].eqNullSafe('1830')|
                            df_inpatient["ICD9_DGNS_CD_4"].eqNullSafe('1830')|
                            df_inpatient["ICD9_DGNS_CD_5"].eqNullSafe('1830')|
                            df_inpatient["ICD9_DGNS_CD_6"].eqNullSafe('1830')|
                            df_inpatient["ICD9_DGNS_CD_7"].eqNullSafe('1830')|
                            df_inpatient["ICD9_DGNS_CD_8"].eqNullSafe('1830')|
                            df_inpatient["ICD9_DGNS_CD_9"].eqNullSafe('1830')|
                            df_inpatient["ICD9_DGNS_CD_10"].eqNullSafe('1830')
                           )

In [162]:
df0000.count()

2649

In [5]:
from pyspark.sql import functions as F        
   

In [167]:
df_inpatient = df_inpatient.withColumn('class', (F.when(((F.col('ICD9_DGNS_CD_1').eqNullSafe('1830'))|
                                                (F.col('ICD9_DGNS_CD_2').eqNullSafe('1830'))|
                                                (F.col('ICD9_DGNS_CD_3').eqNullSafe('1830'))|
                                                (F.col('ICD9_DGNS_CD_4').eqNullSafe('1830'))|
                                                (F.col('ICD9_DGNS_CD_5').eqNullSafe('1830'))|
                                                (F.col('ICD9_DGNS_CD_6').eqNullSafe('1830'))|
                                                (F.col('ICD9_DGNS_CD_7').eqNullSafe('1830'))|
                                                (F.col('ICD9_DGNS_CD_8').eqNullSafe('1830'))|
                                                (F.col('ICD9_DGNS_CD_9').eqNullSafe('1830'))|
                                                (F.col('ICD9_DGNS_CD_10').eqNullSafe('1830'))),1)
                                                .otherwise(0)))

In [173]:
df_inpatient.select('class').distinct().rdd.map(lambda r: r[0]).collect()

[1, 0]

In [180]:
from pyspark.sql import dataframe
df_inpatient.cube('class').count().show()

+-----+-------+
|class|  count|
+-----+-------+
|    1|   2649|
| null|1332822|
|    0|1330173|
+-----+-------+



In [174]:
dfin_oc = df_inpatient.where(df_inpatient['class'] == 1)

In [175]:
dfin_nooc = df_inpatient.where(df_inpatient['class'] == 0)

In [181]:
dfin_oc.count(), dfin_nooc.count()

(2649, 1330173)

In [176]:
dfin_oc.count()+ dfin_nooc.count() == df_inpatient.count()

True

In [9]:
def split_df(df):
    df = df.withColumn('class', (F.when(((F.col('ICD9_DGNS_CD_1').eqNullSafe('1830'))|
                                        (F.col('ICD9_DGNS_CD_2').eqNullSafe('1830'))|
                                        (F.col('ICD9_DGNS_CD_3').eqNullSafe('1830'))|
                                        (F.col('ICD9_DGNS_CD_4').eqNullSafe('1830'))|
                                        (F.col('ICD9_DGNS_CD_5').eqNullSafe('1830'))|
                                        (F.col('ICD9_DGNS_CD_6').eqNullSafe('1830'))|
                                        (F.col('ICD9_DGNS_CD_7').eqNullSafe('1830'))|
                                        (F.col('ICD9_DGNS_CD_8').eqNullSafe('1830'))|
                                        (F.col('ICD9_DGNS_CD_9').eqNullSafe('1830'))|
                                        (F.col('ICD9_DGNS_CD_10').eqNullSafe('1830'))),1)
                                        .otherwise(0)))
    df_1 = df.where(df['class'] == 1)
    df_0 = df.where(df['class'] == 0)
    return df_1, df_0

def write_df(df, datapath):
    df.write.save(datapath, format='parquet')

In [10]:
df_inpatient_oc,df_inpatient_nooc = split_df(df_inpatient)

In [188]:
df_inpatient_oc.count(),df_inpatient_nooc.count()

(2649, 1330173)

In [11]:
ocpath = "/Users/jill/Downloads/2020_Project/EDA/concat_data/inpatient/OC"
noocpath = "/Users/jill/Downloads/2020_Project/EDA/concat_data/inpatient/noOC"
write_df(df_inpatient_oc,ocpath)


In [12]:
write_df(df_inpatient_nooc,noocpath)

In [14]:
## Test new dataset
df1 = spark.read.load("/Users/jill/Downloads/2020_Project/EDA/concat_data/inpatient/OC")
df2 = spark.read.load("/Users/jill/Downloads/2020_Project/EDA/concat_data/inpatient/noOC")

In [15]:
df1.count(),df2.count()

(2649, 1330173)

In [16]:
df1.printSchema()

root
 |-- DESYNPUF_ID: string (nullable = true)
 |-- CLM_ID: long (nullable = true)
 |-- SEGMENT: integer (nullable = true)
 |-- CLM_FROM_DT: integer (nullable = true)
 |-- CLM_THRU_DT: integer (nullable = true)
 |-- PRVDR_NUM: string (nullable = true)
 |-- CLM_PMT_AMT: double (nullable = true)
 |-- NCH_PRMRY_PYR_CLM_PD_AMT: double (nullable = true)
 |-- AT_PHYSN_NPI: long (nullable = true)
 |-- OP_PHYSN_NPI: long (nullable = true)
 |-- OT_PHYSN_NPI: long (nullable = true)
 |-- CLM_ADMSN_DT: integer (nullable = true)
 |-- ADMTNG_ICD9_DGNS_CD: string (nullable = true)
 |-- CLM_PASS_THRU_PER_DIEM_AMT: double (nullable = true)
 |-- NCH_BENE_IP_DDCTBL_AMT: double (nullable = true)
 |-- NCH_BENE_PTA_COINSRNC_LBLTY_AM: double (nullable = true)
 |-- NCH_BENE_BLOOD_DDCTBL_LBLTY_AM: double (nullable = true)
 |-- CLM_UTLZTN_DAY_CNT: integer (nullable = true)
 |-- NCH_BENE_DSCHRG_DT: integer (nullable = true)
 |-- CLM_DRG_CD: string (nullable = true)
 |-- ICD9_DGNS_CD_1: string (nullable = true)


In [17]:
df2.printSchema()

root
 |-- DESYNPUF_ID: string (nullable = true)
 |-- CLM_ID: long (nullable = true)
 |-- SEGMENT: integer (nullable = true)
 |-- CLM_FROM_DT: integer (nullable = true)
 |-- CLM_THRU_DT: integer (nullable = true)
 |-- PRVDR_NUM: string (nullable = true)
 |-- CLM_PMT_AMT: double (nullable = true)
 |-- NCH_PRMRY_PYR_CLM_PD_AMT: double (nullable = true)
 |-- AT_PHYSN_NPI: long (nullable = true)
 |-- OP_PHYSN_NPI: long (nullable = true)
 |-- OT_PHYSN_NPI: long (nullable = true)
 |-- CLM_ADMSN_DT: integer (nullable = true)
 |-- ADMTNG_ICD9_DGNS_CD: string (nullable = true)
 |-- CLM_PASS_THRU_PER_DIEM_AMT: double (nullable = true)
 |-- NCH_BENE_IP_DDCTBL_AMT: double (nullable = true)
 |-- NCH_BENE_PTA_COINSRNC_LBLTY_AM: double (nullable = true)
 |-- NCH_BENE_BLOOD_DDCTBL_LBLTY_AM: double (nullable = true)
 |-- CLM_UTLZTN_DAY_CNT: integer (nullable = true)
 |-- NCH_BENE_DSCHRG_DT: integer (nullable = true)
 |-- CLM_DRG_CD: string (nullable = true)
 |-- ICD9_DGNS_CD_1: string (nullable = true)


In [ ]:
outpath = "/Users/jill/Downloads/2020_Project/EDA/concat_data/outpatient/outpatient_con"


In [ ]:
### Carrier concat data
startpath_carrier = "/Users/jill/Downloads/2020_Project/integrated_data/carrier_claim/"
datapath_carrier_lst = datapath_lst(startpath_carrier)
concat_carrier = concat_df(datapath_carrier_lst)